## Machine Unlearning (MU) for Forget me not algorithm. 

 #### **1: Environment Setup**
 
In this section, we set up our Python environment and install the necessary packages. For reproducibility, it’s best to use a virtual environment.


Instructions:

* Create and activate your virtual environment.

* Install our package `unlearn_diff`.


**Prerequisities**

Ensure conda is installed on your system. You can install Miniconda or Anaconda:

* Miniconda (recommended): https://docs.conda.io/en/latest/miniconda.html

* Anaconda: https://www.anaconda.com/products/distribution

After installing conda, ensure it is available in your PATH by running. You may require to restart the terminal session:

In [ ]:
#create venv if you are using python script.

!python -m venv unlearning-env

After creating your virtual environment, if you're working in a notebook, select the kernel associated with that environment. If you're using a terminal instead, activate the environment with:

`source unlearning-env/bin/activate`

In [ ]:
# Install our packaging unlearn_diff

!pip install unlearn_diff

#### 2. Create unlearn_diff environment 

Use this command to create environment:

`create_env`

In [ ]:
!create_env

#### **3. Downloading the Dataset**

After you install the package, you can use the following commands to download.

1. i2p:

* Sample: 

```bash 
        download_data sample i2p
```

* Full: 
```bash 
       download_data full i2p
```

2. quick_canvas:

* Sample: 

```bash 
     download_data sample quick_canvas
```

* Full: 

```bash 
     download_data full quick_canvas
```

In [ ]:
# downloading sample i2p dataset

!download_data sample i2p

In [ ]:
#downloading sample quick_canvas dataset:

!download_data sample quick_canvas

Once you have downloaded datasets, verify the Downloaded Files.

* ls data/i2p-dataset/sample/

* ls -lh ./data/quick-canvas-dataset/sample/

#### **4. Downloading models**

* compvis: 

```bash 
     download_model compvis
```

* diffuser: 

```bash
     download_model diffuser
```

In [ ]:
# download compvis model

#! download_model compvis 

In [ ]:
# download diffuser model

#! download_model diffuser

#### 5. **Train a Text Inversion**

The default configuration for training is provided by forget_me_not_train_ti_mu. You can run the training with the default settings as follows:

```python
from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu
)
algorithm.run(train_type="train_ti")
```

<br> <br>

**Overriding the Default Configuration**

If you need to override the existing configuration settings, you can specify your custom parameters (such as ckpt_path and raw_dataset_dir) directly when initializing the algorithm. For example:

```python
from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50",
    raw_dataset_dir=(
        "/home/ubuntu/Projects/balaram/packaging/data/quick-canvas-dataset/sample"
    ), 
    steps=10,
    use_sample = True
)
algorithm.run(train_type="train_ti")
```
<br><br><br>


Before performing machine unlearning, make sure to generate the safetensors weights file. This file should be produced as part of the unlearning process so that the subsequent machine unlearning step can use these weights. Once generated, the safetensors file will be
referenced via the 'ti_weights_path' parameter.

In [ ]:
#generate safetensors

from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_ti_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_ti_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50",
    raw_dataset_dir=(
        "data/quick-canvas-dataset/sample"
        # "data/i2p-dataset/sample"
    ), 
    steps=10
)
algorithm.run(train_type="train_ti")

In [ ]:
# machine unlearning using the generated safetensors

from mu.algorithms.forget_me_not.algorithm import ForgetMeNotAlgorithm
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_train_attn_mu,
)

algorithm = ForgetMeNotAlgorithm(
    forget_me_not_train_attn_mu,
    ckpt_path="/home/ubuntu/Projects/UnlearnCanvas/UnlearnCanvas/machine_unlearning/models/diffuser/style50",
    raw_dataset_dir=(
        "data/quick-canvas-dataset/sample"
    ),
    steps=10,
    ti_weights_path="outputs/forget_me_not/ti_models/step_inv_10.safetensors",
    devices = "0"
)
algorithm.run(train_type="train_attn")

#### Evaluation of MU

The evaluation framework is used to assess the performance of models after applying machine unlearning.

In [ ]:
from mu.algorithms.forget_me_not import ForgetMeNotEvaluator
from mu.algorithms.forget_me_not.configs import (
    forget_me_not_evaluation_config
)

evaluator = ForgetMeNotEvaluator(
    forget_me_not_evaluation_config,
    ckpt_path="outputs/forget_me_not/finetuned_models/Abstractionism",  #finetuned model path
    classifier_ckpt_path = "/home/ubuntu/Projects/models/classifier_ckpt_path/style50_cls.pth", 
    reference_dir= "data/quick-canvas-dataset/sample/",
    use_sample = True # to evaluate on sample dataset
)
evaluator.run()

#### **Run Attacks**

Before running attacks, download dataset for attack. Run the following command in terminal.

`generate_attack_dataset --prompts_path data/prompts/prompts.csv --concept i2p_nude --save_path outputs/dataset --num_samples 1`

Here, prompts_path is the path of csv containing prompt, concept is the name of the concept for organizing output file, save_path is the directory where generated images and metadata will be saved, num_samples is the number of images to generate per prompt.

1. **Hard Prompt Attack - Diffuser**

Use the following code if you wish to run the hard prompt attack using diffuser model.

In [ ]:
from mu_attack.configs.nudity import hard_prompt_esd_nudity_P4D_diffusers_config
from mu_attack.execs.attack import MUAttack

def run_attack_for_nudity():

    overridable_params = {
       "task.diffusers_model_name_or_path" : "outputs/forget_me_not/finetuned_models/Abstractionism", #path to the finetuned model
        "task.dataset_path" : "outputs/dataset/i2p_nude", #generated images path using above given command
        "logger.json.root" :"results/hard_prompt_esd_nudity_P4D_abstractionism"
    }

    MUAttack(
        config=hard_prompt_esd_nudity_P4D_diffusers_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_attack_for_nudity()

**Random Attack - Diffuser**

Use the following code if you wish to run the random attack using the Diffuser model:

In [ ]:
from mu_attack.configs.nudity import no_attack_esd_nudity_classifier_diffusers_config
from mu_attack.execs.attack import MUAttack

def run_no_attack_for_nudity():

    overridable_params = {
    "task.diffusers_model_name_or_path" :"outputs/forget_me_not/finetuned_models/Abstractionism",
    "task.dataset_path" : "outputs/dataset/i2p_nude",
    "logger.json.root" : "results/no_attack_esd_nudity_P4D_abstrctionism",
    "attacker.no_attack.dataset_path" : "outputs/dataset/i2p_nude"
    }

    MUAttack(
        config=no_attack_esd_nudity_classifier_diffusers_config,
        **overridable_params
    )

if __name__ == "__main__":
    run_no_attack_for_nudity()

#### Evaluation of MU_attack

Calculate the performance of mu_attack using logs and images generated during attack.


In [ ]:
from mu_attack.configs.evaluation import attack_evaluation_config
from mu_attack.execs.evaluator import MuAttackEvaluator

def main():
    config = attack_evaluation_config
    config.asr.root = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d"
    config.asr.root_no_attack = "results/hard_prompt_esd_nudity_P4D_abstrc/NoAttackEsdNudity"
    config.clip.devices = "0"
    config.clip.image_path = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d/images"
    config.clip.log_path = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d/log.json"
    config.fid.ref_batch_path = "results/hard_prompt_esd_nudity_P4D_abstractionism/P4d/images"
    config.fid.sample_batch_path = "data/i2p/nude"

    # Common output path
    config.output_path = "results/evaluation/results.json"

    evaluator = MuAttackEvaluator(config)

    # Run the evaluation (this will run ASR, CLIP, and FID evaluators)
    results = evaluator.run()

    print("Evaluation Results:",results)

if __name__ == "__main__":
    main()

### Mu Defense (AdvUnlean)

After performing unlearning and attack, we need to perform adversarial unlearning by integrating a soft prompt attack into the training loop. use the following code snippet for advunlearn.



In [ ]:
from mu_defense.algorithms.adv_unlearn.algorithm import AdvUnlearnAlgorithm
from mu_defense.algorithms.adv_unlearn.configs import adv_unlearn_config


def mu_defense():

    mu_defense = AdvUnlearnAlgorithm(
        config=adv_unlearn_config,
        diffusers_model_name_or_path = "/home/ubuntu/Projects/dipesh/unlearn_diff/outputs/forget_me_not/finetuned_models/Abstractionism", #finetuned model
        attack_step = 2,
        backend = "diffusers",
        attack_method = "fast_at",
        # train_method = "text_encoder_full",  #training method. check for docs for available train_method 
        train_method = "noxattn", #training method. check for docs for available train_method 
        warmup_iter = 1,
        iterations = 1
    )
    mu_defense.run()

if __name__ == "__main__":
    mu_defense()


### Evaluation for mu_defense

Before proceeding with evaluation, you must generate images using the output from mu_defense.

To generate images use the following code snippet.


Description of params used:

* config: default train config for image generation.
* target_ckpt: Model ckpt after running mu_defense (AdvUnleran).
* save_path: output dir to save generated images.
* prompts_path: path to the csv with prompts.
* num_samples: number of samples to be generated for a prompt.
* folder_suffix: suffix for folder name for save path.
* devices: devices to be used.  

In [ ]:
from mu_defense.algorithms.adv_unlearn.configs import example_image_generator_config
from mu_defense.algorithms.adv_unlearn import ImageGenerator
from mu.algorithms.erase_diff.configs import erase_diff_train_mu

def generate_image():
    generate_image = ImageGenerator(
        config = example_image_generator_config,
        target_ckpt = "outputs/results_with_retaining/nudity/coco_object/pgd/AttackLr_0.001/text_encoder_full/all/prefix_k/AdvUnlearn-nudity-method_text_encoder_full_all-Attack_pgd-Retain_coco_object_iter_1.0-lr_1e-05-AttackLr_0.001-prefix_k_adv_num_1-word_embd-attack_init_latest-attack_step_30-adv_update_1-warmup_iter_200/",
        save_path = "outputs/adv_unlearn/models",
        prompts_path = "data/prompts/sample_prompts.csv",
        num_samples = 1,
        folder_suffix = "imagenette",
        devices = "0",

    )
    generate_image.generate_images()

if __name__ == "__main__":
    generate_image()

Once the images are generated using the finetuned model from defense, you need to evaulate it's performance: 

In [ ]:
from mu_defense.algorithms.adv_unlearn import MUDefenseEvaluator
from mu_defense.algorithms.adv_unlearn.configs import mu_defense_evaluation_config

def mu_defense_evaluator():
    evaluator = MUDefenseEvaluator(
        config = mu_defense_evaluation_config,
        gen_imgs_path = "outputs/adv_unlearn/models_visualizations_imagenette/SD-v1-4/",
        coco_imgs_path = "coco_dataset/sample",
        output_path = "outputs/adv_unlearn/evaluation/",
        devices = "0"
        # job = "clip", #donot use this if you want to calculate both clip and fid score
    )
    evaluator.run()


if __name__ == "__main__":
    mu_defense_evaluator()